In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Embedding,Activation, Dropout
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D 
import numpy as np
from numpy import array
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv('twitter4000.csv')

text = df['twitts'].tolist()

y = df['sentiment']

token = Tokenizer()
token.fit_on_texts(text)

vocab_size = len(token.word_index) + 1

encoded_text = token.texts_to_sequences(text)

max_length = 120
X = pad_sequences(encoded_text, maxlen=max_length, padding='post')

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.2, stratify = y)

vec_size = 300

# 1DCNN Model
cnn_model = Sequential()
cnn_model.add(Embedding(vocab_size, vec_size, input_length=max_length))
cnn_model.add(Conv1D(64, 8, activation='relu'))
cnn_model.add(MaxPooling1D(2))
cnn_model.add(Dropout(0.2))
cnn_model.add(GlobalMaxPooling1D())
cnn_model.add(Dense(32, activation='relu'))
cnn_model.add(Dropout(0.5))
cnn_model.add(Dense(16, activation='relu'))
cnn_model.add(Dense(1, activation='sigmoid'))

cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
cnn_model.summary()

cnn_model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

# LSTM Model
lstm_model = Sequential()
lstm_model.add(Embedding(vocab_size, vec_size, input_length=max_length))
lstm_model.add(LSTM(128))
lstm_model.add(Dropout(0.5))
lstm_model.add(Dense(32, activation='relu'))
lstm_model.add(Dense(1, activation='sigmoid'))

lstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
lstm_model.summary()

lstm_model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

# BiLSTM Model
bilstm_model = Sequential()
bilstm_model.add(Embedding(vocab_size, vec_size, input_length=max_length))
bilstm_model.add(Bidirectional(LSTM(128)))
bilstm_model.add(Dropout(0.5))
bilstm_model.add(Dense(32, activation='relu'))
bilstm_model.add(Dense(1, activation='sigmoid'))

bilstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
bilstm_model.summary()

bilstm_model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

def get_encoded(x):
    x = token.texts_to_sequences(x)
    x = pad_sequences(x, maxlen=max_length, padding='post')
    return x

# Sentences you want to test
sentences = ['worst services. will not come again', 'thank you for watching']

# Encode sentences once (batch them)
X_sentences = get_encoded(sentences)

# Models to use
models = {
    '1D CNN': cnn_model,
    'LSTM': lstm_model,
    'BiLSTM': bilstm_model
}

# Predict and display
for model_name, model in models.items():
    preds = model.predict(X_sentences)  # Batch predict all sentences at once
    preds = (preds > 0.5).astype("int32").flatten()  # Convert predictions to 0/1

    for sentence, pred in zip(sentences, preds):
        sentiment = 'Positive' if pred == 1 else 'Negative'
        print(f"Model: {model_name} | Sentence: \"{sentence}\" | Prediction: {pred} ({sentiment})")

